<a href="https://colab.research.google.com/github/vamsikrishna00466/stock-market-anlalysis/blob/main/signal_analysis0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#df1.to_excel("sql_cash_data.xlsx")
#Creating a function to convert x min data to y min data.(x<y)
def Change_Timeframe(data, timedelta):
    """
    This function will convert 5min data to 15min data.
    Parameters:
    data: Cleaned data which contains a column named 'DateTime'.
    timedelta: New timeframe in which you want to convert the data to.
    """
    
    
    #We need to sort the index before resampling
    data = data.copy()
    data.set_index(data.DateTime,inplace=True)
    data.sort_index(inplace=True)

    #Creating a Dictionary to apply functions to various columns on resampling
    aggregation_dict = {'script_name':'first',
                        'open': 'first',
                        'high': 'max',
                        'low': 'min',
                        'close': 'last',}

    #Creating a Dictionary to change the names of the respective Columns
    rename_dict = {'open':'Open_15',
                   'high':'High_15',
                   'low':'Low_15',
                   'close':'Close_15',
                   'script_name':'ScriptName'}

    return (data
            .resample(timedelta)
            .agg(aggregation_dict)
            .rename(columns=rename_dict)
            .dropna()
            .reset_index()
    )




def TrueRange(data):
    data = data.copy()
    data["TR"] = np.nan
    for i in range(1,len(data)):
        h = data.loc[i,"High_15"]
        l = data.loc[i,"Low_15"]
        pc = data.loc[i-1,"Close_15"]
        
        x = h-l
        y = abs(h-pc)
        z = abs(l-pc)
        
        TR = max(x,y,z)
        data.loc[i,"TR"] = TR
    
    return data

def average_true_range(data, period=14, drop_tr=True, smoothing="SMA"):
    """
    Average True Range
    :param data: Pandas Dataframe
    :param period: Period for which the ATR needs to be calculated
    :param drop_tr: Whether to drop the TR field
    :param smoothing: Smoothing type - Possible values: RMA, SMA, EMA
    :return: Pandas Dataframe with new column atr_<period>_<smoothing>. Ex: atr_14_sma
    """
    data = data.copy()
    if smoothing == "RMA":
        data['atr_' + str(period) + '_' + str(smoothing)] = data['TR'].ewm(com=period - 1,
                                                                           min_periods=period).mean()
    elif smoothing == "SMA":
        data['atr_' + str(period) + '_' + str(smoothing)] = data['TR'].rolling(window=period).mean()
    elif smoothing == "EMA":
        data['atr_' + str(period) + '_' + str(smoothing)] = data['TR'].ewm(span=period, adjust=False).mean()
    
    if drop_tr:
        data.drop(['TR'], inplace=True, axis=1)
    data = data.round(decimals=2)

    return data


In [17]:
#change timeframe to 5minutes
new_df = Change_Timeframe(df,"5Min")
new_df["Date"] = list(map(lambda x: str(x.date()),new_df.DateTime))
new_df["Time"] = list(map(lambda x: x.time(),new_df.DateTime))


#change data type decimal to float
new_df['Open_15'] = new_df['Open_15'].astype('float')
new_df['High_15'] = new_df['High_15'].astype('float')
new_df['Low_15'] = new_df['Low_15'].astype('float')
new_df['Close_15'] = new_df['Close_15'].astype('float')

#new_df.to_excel("sql_cash_data_5min.xlsx")

new_df = TrueRange(new_df)
ATR_df = average_true_range(data = new_df, period=14, drop_tr=True, smoothing="SMA")
f=4
n=15


df = ATR_df.copy()
#Calculation of SuperTrend
df['Upper Basic']=(df['High_15']+df['Low_15'])/2+(f*df['atr_14_SMA'])
df['Lower Basic']=(df['High_15']+df['Low_15'])/2-(f*df['atr_14_SMA'])
df['Upper Band']=df['Upper Basic']
df['Lower Band']=df['Lower Basic']
for i in range(n,len(df)):
  if df['Close_15'][i-1]<=df['Upper Band'][i-1]:
    df['Upper Band'][i]=min(df['Upper Basic'][i],df['Upper Band'][i-1])
  else:
    df['Upper Band'][i]=df['Upper Basic'][i]    
for i in range(n,len(df)):
  if df['Close_15'][i-1]>=df['Lower Band'][i-1]:
    df['Lower Band'][i]=max(df['Lower Basic'][i],df['Lower Band'][i-1])
  else:
    df['Lower Band'][i]=df['Lower Basic'][i]   
    df['SuperTrend']=np.nan
for i in df['SuperTrend']:
  if df['Close_15'][n-1]<=df['Upper Band'][n-1]:
    df['SuperTrend'][n-1]=df['Upper Band'][n-1]
  elif df['Close_15'][n-1]>df['Upper Band'][i]:
    df['SuperTrend'][n-1]=df['Lower Band'][n-1]
for i in range(n,len(df)):
  if df['SuperTrend'][i-1]==df['Upper Band'][i-1] and df['Close_15'][i]<=df['Upper Band'][i]:
    df['SuperTrend'][i]=df['Upper Band'][i]
  elif  df['SuperTrend'][i-1]==df['Upper Band'][i-1] and df['Close_15'][i]>=df['Upper Band'][i]:
    df['SuperTrend'][i]=df['Lower Band'][i]
  elif df['SuperTrend'][i-1]==df['Lower Band'][i-1] and df['Close_15'][i]>=df['Lower Band'][i]:
    df['SuperTrend'][i]=df['Lower Band'][i]
  elif df['SuperTrend'][i-1]==df['Lower Band'][i-1] and df['Close_15'][i]<=df['Lower Band'][i]:
    df['SuperTrend'][i]=df['Upper Band'][i]
  

df1 = df[['DateTime', 'Date', 'Time', 'Open_15', 'High_15', 'Low_15', 'Close_15', 'SuperTrend']]
df2 =df1.copy()


#donchan channel 50 candles period
k = 50
df2['upper_channel'] = np.nan
df2['lower_channel'] = np.nan
df2['middle_channel'] = np.nan

df2['upper_channel'] = df2['High_15'].rolling(k).max().shift(k-50)
df2['lower_channel'] = df2['Low_15'].rolling(k).min().shift(k-50)
df2['middle_channel'] = ((df2['upper_channel'] + df2['lower_channel'])/2)

#df2.to_excel("super_trend_v1_1.xlsx")

#df2.to_excel("super_trend_v1_1.xlsx")
df21 = df2.copy()

In [21]:
n=0
df3 = df2.dropna(axis=0, inplace=False)
df3=df3.reset_index()
#trade type 
df3['trade_type'] = np.nan
#long position above supertrend and second candle break first candle high
for i in range(n,len(df3)):
  if df3['upper_channel'][i] <= df3['High_15'][i] and (df3.loc[i,"Time"]>=datetime.datetime.strptime('10:00:00', '%H:%M:%S').time()) & (df3.loc[i,"Time"]<datetime.datetime.strptime('14:45:00', '%H:%M:%S').time()):
    df3['trade_type'][i]= 'long'
#long position below supertrend and second candle break first candle low
for i in range(n,len(df3)):
  if df3['lower_channel'][i] >= df3['Low_15'][i] and (df3.loc[i,"Time"]>=datetime.datetime.strptime('10:00:00', '%H:%M:%S').time()) & (df3.loc[i,"Time"]<datetime.datetime.strptime('14:45:00', '%H:%M:%S').time()):
    df3['trade_type'][i]= 'short'

In [23]:
df4 = df3.dropna(axis=0, inplace=False)
df4=df4.reset_index()
df4 = df4.drop(['level_0', 'index'], axis=1)
#df4['trade_type'].value_counts()

In [54]:
df4.tail()

,DateTime,Date,Time,Open_15,High_15,Low_15,Close_15,SuperTrend,upper_channel,lower_channel,middle_channel,sell_signal
67204,2019-10-30 14:20:00,2019-10-30,14:20:00,30021.55,30042.50,30011.05,30018.75,29951.995,30147.8,29944.7,30046.25,long_sale
67205,2019-10-30 14:25:00,2019-10-30,14:25:00,30002.90,30029.15,29990.70,30006.60,29951.995,30147.8,29944.7,30046.25,long_sale
67206,2019-10-30 14:30:00,2019-10-30,14:30:00,30008.00,30009.75,29946.15,29967.80,29951.995,30147.8,29944.7,30046.25,long_sale
67207,2019-10-30 14:35:00,2019-10-30,14:35:00,29969.80,29976.70,29931.80,29940.40,30154.290,30147.8,29931.8,30039.80,long_sale
67208,2019-10-30 14:40:00,2019-10-30,14:40:00,29941.65,30000.85,29917.20,30000.75,30154.290,30147.8,29917.2,30032.50,long_sale


In [44]:
n=0
df3 = df2.dropna(axis=0, inplace=False)
df3=df3.reset_index()

#trade signal 
df3['sell_signal'] = np.nan
#long position above supertrend and second candle break first candle high
for i in range(n,len(df3)):
  if df3['middle_channel'][i] >= df3['Low_15'][i] and (df3.loc[i,"Time"]>=datetime.datetime.strptime('10:00:00', '%H:%M:%S').time()) & (df3.loc[i,"Time"]<datetime.datetime.strptime('14:45:00', '%H:%M:%S').time()):
    df3['sell_signal'][i]= 'long_sale'
#long position below supertrend and second candle break first candle low
for i in range(n,len(df3)):
  if df3['middle_channel'][i] <= df3['High_15'][i] and (df3.loc[i,"Time"]>=datetime.datetime.strptime('10:00:00', '%H:%M:%S').time()) & (df3.loc[i,"Time"]<datetime.datetime.strptime('14:45:00', '%H:%M:%S').time()):
    df3['sell_signal'][i]= 'short_sale'

In [48]:
df4 = df3.dropna(axis=0, inplace=False)
df4=df4.reset_index()
df4 = df4.drop(['level_0', 'index'], axis=1)
#df4['trade_type'].value_counts()

In [49]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67209 entries, 0 to 67208
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DateTime        67209 non-null  datetime64[ns]
 1   Date            67209 non-null  object        
 2   Time            67209 non-null  object        
 3   Open_15         67209 non-null  float64       
 4   High_15         67209 non-null  float64       
 5   Low_15          67209 non-null  float64       
 6   Close_15        67209 non-null  float64       
 7   SuperTrend      67209 non-null  float64       
 8   upper_channel   67209 non-null  float64       
 9   lower_channel   67209 non-null  float64       
 10  middle_channel  67209 non-null  float64       
 11  sell_signal     67209 non-null  object        
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 6.2+ MB


In [51]:
df4['sell_signal'].value_counts()

short_sale    39132
long_sale     28077
Name: sell_signal, dtype: int64

In [55]:
df2.tail()

,DateTime,Date,Time,Open_15,High_15,Low_15,Close_15,SuperTrend,upper_channel,lower_channel,middle_channel
88535,2019-10-30 14:55:00,2019-10-30,14:55:00,29990.25,30021.50,29989.05,30021.50,30154.290,30147.8,29917.2,30032.5
88536,2019-10-30 15:00:00,2019-10-30,15:00:00,30022.30,30023.55,29979.95,29982.20,30154.290,30147.8,29917.2,30032.5
88537,2019-10-30 15:05:00,2019-10-30,15:05:00,29978.65,29984.10,29962.30,29972.25,30143.960,30147.8,29917.2,30032.5
88538,2019-10-30 15:10:00,2019-10-30,15:10:00,29971.75,29987.75,29957.00,29986.90,30134.695,30147.8,29917.2,30032.5
88539,2019-10-30 15:15:00,2019-10-30,15:15:00,29989.40,29990.40,29957.20,29968.45,30134.695,30147.8,29917.2,30032.5


In [52]:
df4.tail()

,DateTime,Date,Time,Open_15,High_15,Low_15,Close_15,SuperTrend,upper_channel,lower_channel,middle_channel,sell_signal
67204,2019-10-30 14:20:00,2019-10-30,14:20:00,30021.55,30042.50,30011.05,30018.75,29951.995,30147.8,29944.7,30046.25,long_sale
67205,2019-10-30 14:25:00,2019-10-30,14:25:00,30002.90,30029.15,29990.70,30006.60,29951.995,30147.8,29944.7,30046.25,long_sale
67206,2019-10-30 14:30:00,2019-10-30,14:30:00,30008.00,30009.75,29946.15,29967.80,29951.995,30147.8,29944.7,30046.25,long_sale
67207,2019-10-30 14:35:00,2019-10-30,14:35:00,29969.80,29976.70,29931.80,29940.40,30154.290,30147.8,29931.8,30039.80,long_sale
67208,2019-10-30 14:40:00,2019-10-30,14:40:00,29941.65,30000.85,29917.20,30000.75,30154.290,30147.8,29917.2,30032.50,long_sale


In [57]:
n=0
df3 = df2.dropna(axis=0, inplace=False)
df3=df3.reset_index()

#trade signal 
df3['sell_signal'] = np.nan
#long position above supertrend and second candle break first candle high
for i in range(n,len(df3)):
  if df3['middle_channel'][i] >= df3['Low_15'][i] or df3['SuperTrend'][i] >= df3['Low_15'][i] and (df3.loc[i,"Time"]>=datetime.datetime.strptime('10:00:00', '%H:%M:%S').time()) & (df3.loc[i,"Time"]<datetime.datetime.strptime('14:45:00', '%H:%M:%S').time()):
    df3['sell_signal'][i]= 'long_sale'
#long position below supertrend and second candle break first candle low
for i in range(n,len(df3)):
  if df3['middle_channel'][i] <= df3['High_15'][i] or df3['SuperTrend'][i] <= df3['Low_15'][i] and (df3.loc[i,"Time"]>=datetime.datetime.strptime('10:00:00', '%H:%M:%S').time()) & (df3.loc[i,"Time"]<datetime.datetime.strptime('14:45:00', '%H:%M:%S').time()):
    df3['sell_signal'][i]= 'short_sale'

In [58]:
df4 = df3.dropna(axis=0, inplace=False)
df4=df4.reset_index()
df4 = df4.drop(['level_0', 'index'], axis=1)
#df4['trade_type'].value_counts()

In [59]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88491 entries, 0 to 88490
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DateTime        88491 non-null  datetime64[ns]
 1   Date            88491 non-null  object        
 2   Time            88491 non-null  object        
 3   Open_15         88491 non-null  float64       
 4   High_15         88491 non-null  float64       
 5   Low_15          88491 non-null  float64       
 6   Close_15        88491 non-null  float64       
 7   SuperTrend      88491 non-null  float64       
 8   upper_channel   88491 non-null  float64       
 9   lower_channel   88491 non-null  float64       
 10  middle_channel  88491 non-null  float64       
 11  sell_signal     88491 non-null  object        
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 8.1+ MB
